## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Airai,-8.9266,125.4092,80.89,34,45,3.15,TL,2021-10-04 03:42:02
1,1,Inhambane,-23.8650,35.3833,73.38,94,75,9.22,MZ,2021-10-04 03:38:57
2,2,Mar Del Plata,-38.0023,-57.5575,46.13,68,0,7.00,AR,2021-10-04 03:32:40
3,3,Necochea,-38.5473,-58.7368,41.59,84,0,7.63,AR,2021-10-04 03:42:03
4,4,Butaritari,3.0707,172.7902,82.81,69,7,16.44,KI,2021-10-04 03:42:03


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria \
min_temp = float(
    input("What is the minimum temperature you would like for your trip? "))
max_temp = float(
    input("What is the maximum temperature you would like for your trip? "))


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Airai,-8.9266,125.4092,80.89,34,45,3.15,TL,2021-10-04 03:42:02
4,4,Butaritari,3.0707,172.7902,82.81,69,7,16.44,KI,2021-10-04 03:42:03
8,8,Faanui,-16.4833,-151.7500,78.82,72,14,10.31,PF,2021-10-04 03:32:32
9,9,Port-Gentil,-0.7193,8.7815,77.36,84,99,12.59,GA,2021-10-04 03:42:05
10,10,Ulladulla,-35.3500,150.4667,76.10,40,1,8.05,AU,2021-10-04 03:42:05
14,14,Matara,5.9485,80.5353,84.02,70,100,10.07,LK,2021-10-04 03:42:06
16,16,Ende,-8.8432,121.6623,86.27,60,77,6.89,ID,2021-10-04 03:42:07
17,17,Atuona,-9.8000,-139.0333,77.88,72,17,17.22,PF,2021-10-04 03:42:07
18,18,Cockburn Town,21.4612,-71.1419,82.54,64,100,20.69,TC,2021-10-04 03:36:39
21,21,Hithadhoo,-0.6000,73.0833,83.39,75,51,16.60,MV,2021-10-04 03:32:10


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       215
City          215
Lat           215
Lng           215
Max Temp      215
Humidity      215
Cloudiness    215
Wind Speed    215
Country       215
Date          215
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Airai,TL,80.89,-8.9266,125.4092,
4,Butaritari,KI,82.81,3.0707,172.7902,
8,Faanui,PF,78.82,-16.4833,-151.7500,
9,Port-Gentil,GA,77.36,-0.7193,8.7815,
10,Ulladulla,AU,76.10,-35.3500,150.4667,
14,Matara,LK,84.02,5.9485,80.5353,
16,Ende,ID,86.27,-8.8432,121.6623,
17,Atuona,PF,77.88,-9.8000,-139.0333,
18,Cockburn Town,TC,82.54,21.4612,-71.1419,
21,Hithadhoo,MV,83.39,-0.6000,73.0833,


In [8]:
from config import g_key

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 10000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_df = hotel_df.dropna()
clean_df


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Airai,TL,80.89,-8.9266,125.4092,Foho Tailau
4,Butaritari,KI,82.81,3.0707,172.7902,Isles Sunset Lodge
8,Faanui,PF,78.82,-16.4833,-151.7500,InterContinental Bora Bora Resort Thalasso Spa
9,Port-Gentil,GA,77.36,-0.7193,8.7815,Fengshui Residence
10,Ulladulla,AU,76.10,-35.3500,150.4667,Sandpiper Motel
...,...,...,...,...,...,...
673,Clarence Town,AU,85.62,-32.5833,151.7833,River Downs Cottage
676,Trincomalee,LK,82.99,8.5711,81.2335,Welcombe Hotel
680,Poum,NC,75.31,-20.2333,164.0167,Gîte kajeon
683,Manokwari,ID,83.07,-0.8667,134.0833,ASTON Niu Manokwari Hotel and Conference Center


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/Weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")


In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = "city name","country code",""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

AttributeError: 'tuple' object has no attribute 'format'

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
